# Fetch Google travel times

In [1]:
import urllib2, json
import pandas as pd
import numpy as np
import datetime, time
from SECRET_KEY import secret_key

In [2]:
# Load TAZ records with x and y coordinates attached
taz = pd.read_csv(r'..\data\taz_xy.txt')

meter_mile_conversion = 0.000621371

- define extraction time (for future travel time predictions)
- set skim as auto

In [161]:
# Create a Google-formatted coordinates field
taz['g_coord'] = taz['y_gps'].astype('str') + ',' + taz['x_gps'].astype('str')

# Look up series of travel times for a single origin

In [173]:
def fetch_times(otaz_list, dtaz_list, dep_time, mode='auto'):
    results = {}
    # Look up 
    for otaz in otaz_list:
        print 'fetching times from zone: ' + str(otaz)

        urlfeed = ""

        origin = taz[taz['ID'] == otaz]['g_coord'].values[0]
        # get list of different destinations
        destination = ''

        for dtaz in dtaz_list:
            if otaz != dtaz:    # skip intrazonal trips where otaz==dtaz
                destination += taz[taz['ID'] == dtaz]['g_coord'].values[0] + '|'
    #             dtaz_list.append(dtaz)
            # remove trailing |
        destination = destination[:-1]

        urlfeed += "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+origin+"&destinations="+destination+ \
                "&mode="+mode+"&departure_time="+dep_time+"&key="+secret_key+"&units=imperial"

        # Fetch url and store
        results[otaz] = json.loads(urllib2.urlopen(urlfeed).read())
        
    return results

In [209]:
def clean_results(results):
    """
    Convert from json to pandas dataframe
    """
    results_list = []
    for origin in otaz_list:
        i = 0
        for dest in results[origin]['destination_addresses']:
            results_dict = {}
            results_dict['origin'] = results[origin]['origin_addresses'][0]
            results_dict['destination'] = dest
            results_dict['distance'] = results[origin]['rows'][0]['elements'][i]['distance']['text']
            results_dict['time_ff'] = results[origin]['rows'][0]['elements'][i]['duration']['text']
            results_dict['time_congested'] = results[origin]['rows'][0]['elements'][i]['duration_in_traffic']['text']
            i+=1
            results_list.append(results_dict)
    
    return pd.DataFrame(results_list)

In [218]:
otaz_list = [532,1552,2570,2286,3108]
dtaz_list = otaz_list

# set standard departure time for tomorrow at 8 AM
dep_hr = 8
dep_time = datetime.datetime.now()

# October 18, current year
dep_time = dep_time.replace(hour=dep_hr,day=18,month=10)
dep_time = str(int(time.mktime(dep_time.timetuple())))

am_results = fetch_times(otaz_list, dtaz_list, dep_time)
df_am = clean_results(am_results)

In [217]:
# PM Peak period
dep_hr = 17
dep_time = datetime.datetime.now()

# October 18, current year
dep_time = dep_time.replace(hour=dep_hr,day=18,month=10)
dep_time = str(int(time.mktime(dep_time.timetuple())))

pm_results = fetch_times(otaz_list, dtaz_list, dep_time)
df_pm = clean_results(pm_results)

In [223]:
df_am['time'] = 'am'
df_pm['time'] = 'pm'
df = df_am.append(df_pm)
df['date'] = '10/18/2017'

In [250]:
df['d_city'] = df['destination'].apply(lambda row: row.split(',')[1].strip())
df['o_city'] = df['origin'].apply(lambda row: row.split(',')[1].strip())

In [262]:
# Look up TAZ
taz_city = {'Seattle':532,
           'Bellevue':1552,
           'Lynnwood':2570,
           'Everett':2286,
           'Tacoma':3108}

df['dtaz'] = df['d_city'].apply(lambda row: taz_city[row])
df['otaz'] = df['o_city'].apply(lambda row: taz_city[row])

In [263]:
df.to_csv(r'C:\Users\Brice\soundcast\scripts\summarize\inputs\network_summary\od_travel_times.csv',index=False)